<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Custom Machine Learning engine

This notebook shows how to log the payload for the model deployed on custom model serving engine using Watson OpenScale python sdk.

### Contents

- [Setup](#setup)
- [OpenScale configuration](#openscale)
- [Performance monitor, scoring and payload logging](#performance)
- [Quality monitor and feedback logging](#quality)
- [Fairness monitoring and explanations](#fairness)
- [Custom monitors and metrics](#custom)
- [Payload analytics](#analytics)

## 1. Setup <a id="setup"></a>

### 1.0 Sample custom machine learning engine

The sample machine learning engine code based on python flask and deployment instructions can be found [here](https://github.com/pmservice/ai-openscale-tutorials/tree/master/applications/custom-ml-engine-bluemix).

Follow the intructions and deploy your own instance of the custom engine application.

**NOTE:** If you use a different CUSTOM machine learning engine, it must follow this [API specification](https://aiopenscale-custom-deployement-spec.mybluemix.net/) to be supported.

### 1.1 Installation and authentication

In [ ]:
!pip install ibm-ai-openscale==2.1.6 --no-cache | tail -n 1

In [ ]:
!pip install pyspark | tail -n 1
!pip install lime | tail -n 1
!pip install pixiedust | tail -n 1

Import and initiate.

In [ ]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.supporting_classes import PayloadRecord
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
import pandas as pd

#### ACTION: Get Watson OpenScale `instance_guid` and `apikey`

You will use an instance of [Watson OpenScale](https://console.bluemix.net/catalog/services/ai-openscale).

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using bluemix console:

- This will contain an `API Key` used as `apikey` below

```bash
ibmcloud login --sso
ibmcloud iam api-key-create 'my_key'
```

How to get your Watson OpenScale instance GUID:

- If your resource group is different than `default`, switch to the resource group containing your Watson OpenScale instance
```bash
ibmcloud target -g <myResourceGroup>
```
- get details of the instance. This contains the GUID used as `instance_guid` below
```bash
ibmcloud resource service-instance <Watson-OpenScale-instance_name>
```

#### Let's define some constants required to set up data mart:

- AIOS_CREDENTIALS
- POSTGRES_CREDENTIALS
- SCHEMA_NAME

In [ ]:
AIOS_CREDENTIALS = {
  "url": "https://api.aiopenscale.cloud.ibm.com",
  "instance_guid": "***",
  "apikey": "***"
}

In [ ]:
POSTGRES_CREDENTIALS = {
    "db_type": "postgresql",
    "uri_cli_1": "xxx",
    "maps": [],
    "instance_administration_api": {
        "instance_id": "xxx",
        "root": "xxx",
        "deployment_id": "xxx"
    },
    "name": "xxx",
    "uri_cli": "xxx",
    "uri_direct_1": "xxx",
    "ca_certificate_base64": "xxx",
    "deployment_id": "xxx",
    "uri": "xxx"
}

In [ ]:
SCHEMA_NAME = 'custom_ml_engine'

Create schema for data mart.

In [ ]:
create_postgres_schema(postgres_credentials=POSTGRES_CREDENTIALS, schema_name=SCHEMA_NAME)

In [ ]:
client = APIClient(AIOS_CREDENTIALS)

In [ ]:
client.version

### 1.2 Training data

#### Get training data set from git

In [ ]:
!rm credit_risk_training.csv 
!wget https://raw.githubusercontent.com/IBM/monitor-custom-ml-engine-with-watson-openscale/master/data/credit_risk_training.csv

#### Preview data

In [ ]:
data_df = pd.read_csv("credit_risk_training.csv",
                    dtype={'LoanDuration': int, 'LoanAmount': int, 'InstallmentPercent': int, 'CurrentResidenceDuration': int, 'Age': int, 'ExistingCreditsCount': int, 'Dependents': int})

In [ ]:
data_df

## 2. OpenScale configuration <a name="openscale"></a>

### 2.1 DataMart setup

>NOTE: If you have already created a data_mart and need to delete it, uncomment and run the cell below:

In [ ]:
#client.data_mart.delete()

In [ ]:
client.data_mart.setup(db_credentials=POSTGRES_CREDENTIALS, schema=SCHEMA_NAME)

In [ ]:
data_mart_details = client.data_mart.get_details()

### 2.2 Bind machine learning engines

#### 2.2.1 Bind  `CUSTOM` machine learning engine
**NOTE:** CUSTOM machine learning engine must follow this [API specification](https://aiopenscale-custom-deployement-spec.mybluemix.net/) to be supported.

Credentials support following fields:
- `url` - hostname and port (required)
- `username` - part of BasicAuth (optional)
- `password` - part of BasicAuth (optional)

In [ ]:
CUSTOM_ENGINE_CREDENTIALS = {
    "url": "****"
}
# OR if you have BasicAuth use:
"""
CUSTOM_ENGINE_CREDENTIALS = {
    "url": "***",
    "username": "***",
    "password": "***"
}
"""

In [ ]:
binding_uid = client.data_mart.bindings.add('My custom engine', CustomMachineLearningInstance(CUSTOM_ENGINE_CREDENTIALS))

In [ ]:
bindings_details = client.data_mart.bindings.get_details()

In [ ]:
client.data_mart.bindings.list()

### 2.3 Subscriptions

#### Add subscriptions

List available deployments.

In [ ]:
client.data_mart.bindings.list_assets()

Let's specify training data information like: list of features and list of categorical features required by fairness and explain.

In [ ]:
feature_columns = ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker']
categorical_columns = ['CheckingStatus', 'CreditHistory', 'LoanPurpose', 'ExistingSavings', 'EmploymentDuration', 'Sex', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans','Housing', 'Job', 'Telephone', 'ForeignWorker']

In [ ]:
subscription = client.data_mart.subscriptions.add(
    CustomMachineLearningAsset(
                source_uid='credit',
                label_column='Risk',
                prediction_column='prediction',
                probability_column='probability',
                feature_columns=feature_columns.copy(),
                categorical_columns=categorical_columns.copy(),
                binding_uid=binding_uid))

#### Get subscriptions list

In [ ]:
subscriptions = client.data_mart.subscriptions.get_details()

In [ ]:
subscriptions_uids = client.data_mart.subscriptions.get_uids()
print(subscriptions_uids)

#### List subscriptions

In [ ]:
client.data_mart.subscriptions.list()

## 3. Performance metrics, scoring and payload logging <a id="performance"></a>

### 3.1 Score the credit model

In [ ]:
import requests
import time

values = [
            ["no_checking", 13, "credits_paid_to_date", "car_new", 1343, "100_to_500", "1_to_4", 2, "female", "none", 3,
             "savings_insurance", 25, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["no_checking", 24, "prior_payments_delayed", "furniture", 4567, "500_to_1000", "1_to_4", 4, "male", "none",
             4, "savings_insurance", 60, "none", "free", 2, "management_self-employed", 1, "none", "yes"],
            ["0_to_200", 26, "all_credits_paid_back", "car_new", 863, "less_100", "less_1", 2, "female", "co-applicant",
             2, "real_estate", 38, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["0_to_200", 14, "no_credits", "car_new", 2368, "less_100", "1_to_4", 3, "female", "none", 3, "real_estate",
             29, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["0_to_200", 4, "no_credits", "car_new", 250, "less_100", "unemployed", 2, "female", "none", 3,
             "real_estate", 23, "none", "rent", 1, "management_self-employed", 1, "none", "yes"],
            ["no_checking", 17, "credits_paid_to_date", "car_new", 832, "100_to_500", "1_to_4", 2, "male", "none", 2,
             "real_estate", 42, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["no_checking", 50, "outstanding_credit", "appliances", 5696, "unknown", "greater_7", 4, "female",
             "co-applicant", 4, "unknown", 54, "none", "free", 2, "skilled", 1, "yes", "yes"],
            ["0_to_200", 13, "prior_payments_delayed", "retraining", 1375, "100_to_500", "4_to_7", 3, "male", "none", 3,
             "real_estate", 70, "none", "own", 2, "management_self-employed", 1, "none", "yes"]
        ]


request_data = {'fields': feature_columns, 'values': values}

header = {'Content-Type': 'application/json'}
scoring_url = subscription.get_details()['entity']['deployments'][0]['scoring_endpoint']['url']

start_time = time.time()
response = requests.post(scoring_url, json=request_data, headers=header)
response_time = int((time.time() - start_time)*1000)

response_data = response.json()
print('Response: ' + str(response_data))

### 3.2 Store the request and response in payload logging table

#### Using Python SDK

**Hint:** You can embed payload logging code into your custom deployment so it is logged automatically each time you score the model.

In [ ]:
records_list = [PayloadRecord(request=request_data, response=response_data, response_time=response_time), 
                PayloadRecord(request=request_data, response=response_data, response_time=response_time)]

for i in range(1, 10):
    records_list.append(PayloadRecord(request=request_data, response=response_data, response_time=response_time))

subscription.payload_logging.store(records=records_list)

#### Using REST API

Get the token first.

In [ ]:
token_endpoint = "https://iam.bluemix.net/identity/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json"
}

data = {
    "grant_type":"urn:ibm:params:oauth:grant-type:apikey",
    "apikey":AIOS_CREDENTIALS["apikey"]
}

req = requests.post(token_endpoint, data=data, headers=headers)
token = req.json()['access_token']

Store the payload.

In [ ]:
import requests, uuid

PAYLOAD_STORING_HREF_PATTERN = '{}/v1/data_marts/{}/scoring_payloads'
endpoint = PAYLOAD_STORING_HREF_PATTERN.format(AIOS_CREDENTIALS['url'], AIOS_CREDENTIALS['data_mart_id'])

payload = [{
    'binding_id': binding_uid, 
    'deployment_id': subscription.get_details()['entity']['deployments'][0]['deployment_id'], 
    'subscription_id': subscription.uid, 
    'scoring_id': str(uuid.uuid4()), 
    'response': response_data,
    'request': request_data
}]


headers = {"Authorization": "Bearer " + token}
      
req_response = requests.post(endpoint, json=payload, headers = headers)

print("Request OK: " + str(req_response.ok))

### 3.3 Performance metrics of scoring requests

In [ ]:
subscription.performance_monitoring.show_table()

## 4. Feedback logging & quality (accuracy) monitoring <a id="quality"></a>

### 4.1 Enable quality monitoring

You need to provide the monitoring `threshold` and `min_records` (minimal number of feedback records).

In [ ]:
subscription.quality_monitoring.enable(threshold=0.8, min_records=10)

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

You can check the schema of feedback table using below method.

In [ ]:
subscription.feedback_logging.print_table_schema()

The feedback records can be send to feedback table using below code.

In [ ]:
records = [
    ["no_checking","28","outstanding_credit","appliances","5990","500_to_1000","greater_7","5","male","co-applicant","3","car_other","55","none","free","2","skilled","2","yes","yes","Risk"],
    ["greater_200","22","all_credits_paid_back","car_used","3376","less_100","less_1","3","female","none","2","car_other","32","none","own","1","skilled","1","none","yes","No Risk"],
    ["no_checking","39","credits_paid_to_date","vacation","6434","unknown","greater_7","5","male","none","4","car_other","39","none","own","2","skilled","2","yes","yes","Risk"],
    ["0_to_200","20","credits_paid_to_date","furniture","2442","less_100","unemployed","3","female","none","1","real_estate","42","none","own","1","skilled","1","none","yes","No Risk"],
    ["greater_200","4","all_credits_paid_back","education","4206","less_100","unemployed","1","female","none","3","savings_insurance","27","none","own","1","management_self-employed","1","none","yes","No Risk"],
    ["greater_200","23","credits_paid_to_date","car_used","2963","greater_1000","greater_7","4","male","none","4","car_other","46","none","own","2","skilled","1","none","yes","Risk"],
    ["no_checking","31","prior_payments_delayed","vacation","2673","500_to_1000","1_to_4","3","male","none","2","real_estate","35","stores","rent","1","skilled","2","none","yes","Risk"],
    ["no_checking","37","prior_payments_delayed","other","6971","500_to_1000","1_to_4","3","male","none","3","savings_insurance","54","none","own","2","skilled","1","yes","yes","Risk"],
    ["0_to_200","39","prior_payments_delayed","appliances","5685","100_to_500","1_to_4","4","female","none","2","unknown","37","none","own","2","skilled","1","yes","yes","Risk"],
    ["no_checking","38","prior_payments_delayed","appliances","4990","500_to_1000","greater_7","4","male","none","4","car_other","50","bank","own","2","unemployed","2","yes","yes","Risk"]]

fields = feature_columns.copy()
fields.append('Risk')

subscription.feedback_logging.store(feedback_data=records, fields=fields)

### 4.2 Run quality monitoring on demand

By default, quality monitoring is run on hourly schedule. You can also trigger it on demand using below code.

In [ ]:
run_details = subscription.quality_monitoring.run(background_mode=False)

### Show the quality metrics

In [ ]:
subscription.quality_monitoring.show_table()

Get all calculated metrics.

In [ ]:
subscription.quality_monitoring.get_metrics(deployment_uid='action')

#### Get metrics as pandas dataframe

In [ ]:
quality_pd = subscription.quality_monitoring.get_table_content(format='pandas')
quality_pd

Create a bar plot.

In [ ]:
%matplotlib inline

quality_pd.plot.barh(x='id', y='value');

## 5. Fairness monitoring and explanations <a id="fairness"></a>

### 5.1 Enable and run fairness monitoring

In [ ]:
from ibm_ai_openscale.supporting_classes import Feature

subscription.fairness_monitoring.enable(
            features=[
                Feature("Sex", majority=['male'], minority=['female'], threshold=0.95),
                Feature("Age", majority=[[26, 75]], minority=[[18, 25]], threshold=0.95)
            ],
            favourable_classes=['No Risk'],
            unfavourable_classes=['Risk'],
            min_records=4,
            training_data=data_df
        )

In [ ]:
fairness_run = subscription.fairness_monitoring.run(background_mode=False)

### 5.2 Check fairness run results

In [ ]:
subscription.fairness_monitoring.show_table()

### 5.3 Explainability configuration and run

#### Enable explainability

In [ ]:
subscription.explainability.enable(training_data=data_df)

#### Get sample transaction_id from payload logging table (`scoring_id`)

In [ ]:
transaction_id = subscription.payload_logging.get_table_content(limit=1)['scoring_id'].values[0]

print(transaction_id)

#### Run explanation for sample `transaction_id`

In [ ]:
explain_run = subscription.explainability.run(transaction_id=transaction_id, background_mode=False)
        

In [ ]:
explain_result = pd.DataFrame.from_dict(explain_run['entity']['predictions'][0]['explanation_features'])
explain_result.plot.barh(x='feature_name', y='weight', color='g', alpha=0.8);

## 6.0 Custom monitoring <a id="custom"></a>

### 6.1 Register custom monitor

In [ ]:
from ibm_ai_openscale.supporting_classes import Metric, Tag

metrics = [Metric(name='sensitivity', lower_limit_default=0.8), Metric(name='specificity', lower_limit_default=0.75)]
tags = [Tag(name='region', description='customer geographical region')]

my_monitor = client.data_mart.monitors.add(name='model performance', metrics=metrics, tags=tags)

In [ ]:
client.data_mart.monitors.list()

#### Get `monitor_uid`

In [ ]:
monitor_uid = my_monitor['metadata']['guid']

print(monitor_uid)

#### Get monitor details

In [ ]:
my_monitor = client.data_mart.monitors.get_details(monitor_uid=monitor_uid)
print('monitor definition details', my_monitor)

### 6.2 Enable custom monitor for subscription

In [ ]:
from ibm_ai_openscale.supporting_classes import Threshold

thresholds = [Threshold(metric_uid='sensitivity', lower_limit=0.9)]
subscription.monitoring.enable(monitor_uid=monitor_uid, thresholds=thresholds)

#### Get custom monitoring configuration details

In [ ]:
subscription.monitoring.get_details(monitor_uid=monitor_uid)

### 6.3 Storing custom metrics

In [ ]:
metrics = {"specificity": 0.78, "sensitivity": 0.67, "region": "us-south"}

subscription.monitoring.store_metrics(monitor_uid=monitor_uid, metrics=metrics)

#### List and get custom metrics

In [ ]:
subscription.monitoring.show_table(monitor_uid=monitor_uid)

In [ ]:
custom_metrics = subscription.monitoring.get_metrics(monitor_uid=monitor_uid, deployment_uid='credit')

In [ ]:
custom_metrics

In [ ]:
custom_metrics_pandas = subscription.monitoring.get_table_content(monitor_uid=monitor_uid)

%matplotlib inline
custom_metrics_pandas.plot.barh(x='id', y='value');

## 7.0 Payload analytics <a id="analytics"></a>

### 7.1 Run data distributions calculation

In [ ]:
from datetime import datetime

start_date = "2018-01-01T00:00:00.00Z"
end_date = datetime.utcnow().isoformat() + "Z"

sex_distribution = subscription.payload_logging.data_distribution.run(
            start_date=start_date,
            end_date=end_date,
            group=['prediction', 'Sex'],
            agg=['count'])

### 7.2 Get data distributions as pandas dataframe

In [ ]:
sex_distribution_run_uid = sex_distribution['id']

In [ ]:
distributions_pd = subscription.payload_logging.data_distribution.get_run_result(run_id=sex_distribution_run_uid, format='pandas')
distributions_pd

### 7.3 Visualize

In [ ]:
subscription.payload_logging.data_distribution.show_chart(sex_distribution_run_uid);

In [ ]:
credit_history_distribution = subscription.payload_logging.data_distribution.run(
            start_date=start_date,
            end_date=end_date,
            group=['prediction', 'CreditHistory'],
            agg=['count'])

In [ ]:
credit_history_distribution_run_uid = credit_history_distribution['id']

subscription.payload_logging.data_distribution.show_chart(credit_history_distribution_run_uid);

---

## Congratulations!

You have finished the tutorial for IBM Watson OpenScale and Azure Machine Learning Studio. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

---

### Authors
Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.